베르노이 나이브 베이즈를 활용한 스팸 메일 분류
이메일 제목과 레이블(스팸 여부) 데이터를 활용해 베르누이 나이브 베이즈 분류로 스팸 메일을 분류한다.

In [8]:
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 베르누이 나이브 베이즈를 사용하기 위한 라이브러리
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
# 모델 정확도 평가를 위한 라이브러리를 import 한다.
from sklearn.metrics import accuracy_score



데이터 획득
간단한 스팸 메일 분류를 위해 다음과 같이 이메일 제목과 스팸 메일 레이블이 있는 데이터를 사용한다.
email title: 이메일 제목, spam: 스팸 메일 여부(True => 스팸 메일, False => 스팸 메일 아님)

In [4]:
email_list = [
    {'email title' : 'free game only today', 'spam' : True},
    {'email title' : 'cheapest flight deal', 'spam' : True},
    {'email title' : 'limited time offer only today only today', 'spam' : True},
    {'email title' : 'today meeting schedule', 'spam' : False},
    {'email title' : 'your flight schedule attached', 'spam' : False},
    {'email title' : 'your credit card statement', 'spam' : False}
]
df = pd.DataFrame(email_list)
df

email title   spam
0                      free game only today   True
1                      cheapest flight deal   True
2  limited time offer only today only today   True
3                    today meeting schedule  False
4             your flight schedule attached  False
5                your credit card statement  False

학습 데이터 다듬기
사이킷런의 베르누이 나이브 베이즈 분류기는 숫자만 다루기 때문에 True와 False는 1과 0으로 치환한다.
이메일 제목(email title)으로 학습을 진행하고 레이블은 label을 사용해서 스팸 메일 여부를 판단한다.

In [5]:
df['label'] = df.get('spam').map({True: 1, False: 0})
df

email title   spam  label
0                      free game only today   True      1
1                      cheapest flight deal   True      1
2  limited time offer only today only today   True      1
3                    today meeting schedule  False      0
4             your flight schedule attached  False      0
5                your credit card statement  False      0

In [6]:
# 학습에 사용할 데이터와 레이블(클래스, 타겟, 답)로 분리한다.
df_x = df['email title']
df_x

0                        free game only today
1                        cheapest flight deal
2    limited time offer only today only today
3                      today meeting schedule
4               your flight schedule attached
5                  your credit card statement
Name: email title, dtype: object

In [7]:
df_y = df['label']
df_y

0    1
1    1
2    1
3    0
4    0
5    0
Name: label, dtype: int64

베르누이 나이브 베이즈의 입력 데이터는 고정된 크기의 벡터여야 한다.
사이킷런의 CountVectorizer() 함수를 사용하면 데이터에 출현한 모든 단어의 개수만큼의 크기를 가진 벡터를 만들고 고정된 벡터로 표현한다.

In [10]:
# CountVectorizer() 함수는 이메일 제목에 출현한 단어를 오름차순으로 정렬해 단어의 위치로 행렬을 만들어 리턴한다.
# 특정 단어가 출현할 경우 출현한 단어의 개수를 리턴하고 출현하지 않으면 0을 리턴한다.
# CountVectorizer() 함수의 옵션으로 binary=True를 지정하면 같은 단어가 여러 번 출현하더라고 무조건 1을 리턴한다.
cv = CountVectorizer()
x_train = cv.fit_transform(df_x)
encoded_input = x_train.toarray() # 넘파이 타입의 배열 데이터로 변환한다.

위의 행렬에서 볼 수 있듯이 이메일 제목에서 총 17개의 단어가 발견되어 각 이메일 제목이 17개 크기의 벡터로 표현(인코딩)된 것을 확인할 수 있다.
베르누이 나이브 베이즈에서 사용하기 위해 이메일 제목에 중복되는 단어가 있더라도 출현한 횟수로 표현되는 것이 아니라 단순히 1로 표현한 것을 알 수 있다.

In [13]:
# inverse_transform() 함수로 고정된 크기의 벡터에 포함되는 단어를 확인할 수 있다.
print(cv.inverse_transform(encoded_input))

[array(['free', 'game', 'only', 'today'], dtype='<U9'), array(['cheapest', 'deal', 'flight'], dtype='<U9'), array(['limited', 'offer', 'only', 'time', 'today'], dtype='<U9'), array(['meeting', 'schedule', 'today'], dtype='<U9'), array(['attached', 'flight', 'schedule', 'your'], dtype='<U9'), array(['card', 'credit', 'statement', 'your'], dtype='<U9')]


In [14]:
# get_feature_names(): 고정된 벡터의 각 열(피쳐)이 어떤 단어를 의미하는지 확인한다.
print(cv.get_feature_names())

['attached', 'card', 'cheapest', 'credit', 'deal', 'flight', 'free', 'game', 'limited', 'meeting', 'offer', 'only', 'schedule', 'statement', 'time', 'today', 'your']


베르누이 나이브 베이즈 모델 학습하기
사이킷런의 베르누이 나이브 베이즈 분류기는 기본적으로 스무딩을 지원하므로 학습 데이터에 없던 단어가 테스트 데이터에 있어도 분류가 잘 진행된다.

라플라스 스무딩(Laplace Smoothing)
0이라는 수는 곱셈과 나눗셈을 무력화시키는 값이므로 그 전에 아무리 의미있는 값이 도출된다 하더라도 마지막에 0을 곱해버리면 0이 나온다. 따라서 값이 0이 아닌 최소값(1회 등장)으로 보정을 하게 되는데 이를 라플라스 스무딩이라 한다.

In [21]:
bnb  = BernoulliNB()
y_train = df_y.astype('int')
bnb.fit(x_train, y_train)

BernoulliNB()

테스트 데이터를 다듬고 테스트 실행

In [19]:
# 테스트 데이터
test_email_list = [
    {'email title' : 'free flight offer', 'spam' : True},
    {'email title' : 'hey traveler free flight deal', 'spam' : True},
    {'email title' : 'limited free game offer', 'spam' : True},
    {'email title' : 'today flight schedule', 'spam' : False},
    {'email title' : 'your credit card attached', 'spam' : False},
    {'email title' : 'free credit card offer only today', 'spam' : False}
]
test_df = pd.DataFrame(email_list)
test_df['label'] = test_df.spam.map({True:1, False:0})
test_df

email title   spam  label
0                      free game only today   True      1
1                      cheapest flight deal   True      1
2  limited time offer only today only today   True      1
3                    today meeting schedule  False      0
4             your flight schedule attached  False      0
5                your credit card statement  False      0

In [23]:
test_x = test_df['email title']
test_x
test_y = test_df['label']
test_y

0    1
1    1
2    1
3    0
4    0
5    0
Name: label, dtype: int64

In [24]:
x_test = cv.transform(test_x)
y_test = test_y.astype('int')

In [25]:
predict = bnb.predict(x_test)
print(classification_report(y_test, predict))

NameError: name 'classification_report' is not defined